### Mencoba membuat RAG lokal dengan llama2 70B diatas laptop core I5.

Kita buat file .env untuk simpan credential API dll selanjutnya declare model, tak hanya untuk llama2 tapi juga model OpenAI (melalui API) sebagai pembanding output <br>

In [ ]:
import os
from dotenv import load_dotenv

MODEL = "llama2"

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_KEY")
# MODEL = "gpt-3.5-turbo"

# MODEL = "mixtral:8x7b"


In [ ]:
#from langchain_openai.chat_models import ChatOpenAI
# from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings


# model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

# model.invoke("Ini percobaan model, beri selamat barangkali?")

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

In [ ]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Jhony Iskandar", "question": "What's your name'?"})

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./pengumuman.pdf")
pages = loader.load_and_split()
pages

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("pengumuman")